In [1]:
import os
import sys
import importlib.util
from tensorflow.keras.models import load_model
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import Sequential
from PIL import Image
from IPython.display import IFrame, display
import numpy as np
import io
import pickle

# Step 1: Dynamic imports
def import_custom_module(name, path):
    try:
        spec = importlib.util.spec_from_file_location(name, path)
        module = importlib.util.module_from_spec(spec)
        spec.loader.exec_module(module)
        print(f"Module '{name}' imported successfully.")
        return module
    except Exception as e:
        print(f"An error occurred when trying to import '{name}':", e)
        return None

test_package_folder_path = os.path.abspath('../Test_package_folder')
image_preprocessing = import_custom_module("image_preprocessing", os.path.join(test_package_folder_path, "image_preprocessing.py"))
emotion_to_track_mvp = import_custom_module("emotion_to_track_mvp", os.path.join(test_package_folder_path, "emotion_to_track_mvp.py"))

# Step 2: Loading models
try:
    pickled_model = pickle.load(open('../models/best_model.pkl', 'rb'))
    print("Pickled model loaded successfully.")
except Exception as e:
    print(f"An error occurred while loading the pickled model: {e}")

# Reconstruct the VGG16-based model
vgg16_base = VGG16(weights='imagenet', include_top=False, input_shape=(150, 150, 3))
vgg16_base.trainable = False

model = Sequential([
    vgg16_base,
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(2, activation='softmax')
])

model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

try:
    model.load_weights('../models/happy_sad_mod.h5')
    print("Model loaded successfully")
except Exception as e:
    print(f"An error occurred while loading the model: {e}")

try:
    model.summary()
except NameError as e:
    print(f"NameError: {e}")
except AttributeError as e:
    print(f"AttributeError: {e}")

# Defining functions for the workflow
def upload_and_preprocess_image(image_path):
    # Assume image_preprocessing function exists and is correct
    processed_img = image_preprocessing.image_preprocessing(image_path)
    return processed_img

def predict_emotion(processed_img):
    # Assume predictions from Keras model
    processed_img = np.expand_dims(processed_img, axis=0)  # Model expects images in batches
    predictions = model.predict(processed_img)
    emotion = 'happy' if predictions[0][0] > 0.5 else 'sad'
    return emotion

def get_song_recommendation(emotion):
    song_embed_code = emotion_to_track_mvp.get_random_track_embed_code(emotion)
    return song_embed_code

def display_in_notebook(emotion, song_embed_code):
    display(IFrame(song_embed_code, width=300, height=380))
    print(f"Detected Emotion: {emotion} and associated song url: {song_embed_code}")

def main_workflow(image_path):
    processed_img = upload_and_preprocess_image(image_path)
    emotion = predict_emotion(processed_img)
    song_embed_code = get_song_recommendation(emotion)
    display_in_notebook(emotion, song_embed_code)

Module 'image_preprocessing' imported successfully.
Module 'emotion_to_track_mvp' imported successfully.
Pickled model loaded successfully.
An error occurred while loading the model: Layer count mismatch when loading weights from file. Model expected 1 layers, found 3 saved layers.


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ vgg16 (Functional)              │ ?                      │    14,714,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,714,688 (56.13 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 14,714,688 (56.13 MB)

In [2]:
# Test Image 1
image_path_1 = "/Users/jbm/Dropbox/Mac/Downloads/Test_Images_Doppelganger/cd8cee459c5e645ae75e8db69ef3d72d.jpg"
main_workflow(image_path_1)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 255ms/step


Detected Emotion: sad and associated song url: <iframe src="https://open.spotify.com/embed/track/1odExI7RdWc4BT515LTAwj" width="300" height="380" frameborder="0" allowtransparency="true" allow="encrypted-media"></iframe>


In [3]:
# Test Image 2
image_path_2 = "/Users/jbm/Dropbox/Mac/Downloads/Test_Images_Doppelganger/3b4b123c8a38cd2a18a12adb2ab9ff96.jpg"
main_workflow(image_path_2)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step


Detected Emotion: sad and associated song url: <iframe src="https://open.spotify.com/embed/track/6wf7Yu7cxBSPrRlWeSeK0Q" width="300" height="380" frameborder="0" allowtransparency="true" allow="encrypted-media"></iframe>
